# Import

In [1]:
# System
import os
import time
import json

# Below blocks should be executed before any modules that with computational cost
# Processor Setting
os.environ["OMP_NUM_THREADS"] = "4" # export OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "4" # export OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "6" # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "4" # export VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "6" # export NUMEXPR_NUM_THREADS=6

# Data processing
import numpy as np
import netCDF4 as nc

# Config

In [2]:
# Config path
root = '/Volumes/Expansion/User_Backup/b08209033/111-2_IVT_analysis/'
file = 'config.json'
config_path = os.path.join(root, file)

# Import config
with open(config_path) as infile:
    config = json.load(infile)
    infile.close()

# Update config
    # IVT variable
config.update({"Path_IVT_calculation": os.path.join(config["Path_data"], "IVT")})
config.update({"Fname_RAW_IVT_original": "dataset.nc"})
config.update({"Fname_RAW_IVT_seasonal": "seasonal_cycle.nc"})
config.update({"Fname_IVT": "IVT_raw.npz"})

# Export config
with open(config_path, 'w') as outfile:
    json.dump(config, outfile, sort_keys=True)
    outfile.close()

# IVT

In [3]:
def getIVT(original, seasonal = None):
    """
    Description:
        Calculate IVT from .nc file
    Input:
        Require:
            filename, .nc object
        Optional:
            filename, .nc object
    Output:
        Require:
            lat
            lon
            Original IVT [year, day, lat, lon]
        Optional:
            Seasonal IVT [day, lat, lon]
    """
    # Original IVT
    with nc.Dataset(original) as dataset:
        # Variable dimension
        lon       = dataset['lon'][:]
        lat       = dataset['lat'][:]
        level     = dataset['plev'][:]
        thickness = (level[:-1] - level[1:])[np.newaxis, np.newaxis, :, np.newaxis, np.newaxis]
        # Variable
        u = dataset['u'][:,:,:,:].reshape(-1, 365, level.shape[0], lat.shape[0], lon.shape[0])
        v = dataset['v'][:,:,:,:].reshape(-1, 365, level.shape[0], lat.shape[0], lon.shape[0])
        q = dataset['q'][:,:,:,:].reshape(-1, 365, level.shape[0], lat.shape[0], lon.shape[0])
        # Calculate
    u = (u[:,:,1:,:,:] + u[:,:,:-1,:,:])/2
    v = (v[:,:,1:,:,:] + v[:,:,:-1,:,:])/2
    q = (q[:,:,1:,:,:] + q[:,:,:-1,:,:])/2 
    
    IVT_x_original = np.sum((u * q * thickness)/9.81, axis = 2)
    IVT_y_original = np.sum((v * q * thickness)/9.81, axis = 2)
    IVT_original = np.array([IVT_x_original, IVT_y_original])
    
    # Seasonal IVT
    if (seasonal != None):
        with nc.Dataset(seasonal) as dataset:
        # Variable dimension
            lon       = dataset['lon'][:]
            lat       = dataset['lat'][:]
            level     = dataset['plev'][:]
            thickness = (level[:-1] - level[1:])[np.newaxis, :, np.newaxis, np.newaxis]
        # Variable
            u = dataset['u'][:,:,:,:]
            v = dataset['v'][:,:,:,:]
            q = dataset['q'][:,:,:,:]
        # Calculate
        u = (u[:,1:,:,:] + u[:,:-1,:,:])/2
        v = (v[:,1:,:,:] + v[:,:-1,:,:])/2
        q = (q[:,1:,:,:] + q[:,:-1,:,:])/2
        
        IVT_x_seasonal = np.sum((u * q * thickness)/9.81, axis = 1)
        IVT_y_seasonal = np.sum((v * q * thickness)/9.81, axis = 1)
        IVT_seasonal = np.array([IVT_x_seasonal, IVT_y_seasonal])
        
        return lat, lon, IVT_original, IVT_seasonal
    
    return lat, lon, IVT_original

In [5]:
# Calculate IVT
count = time.time()
os.chdir(config["Path_data"])
original = "dataset.nc"
seasonal = "seasonal_cycle.nc"
lat, lon, IVT_original, IVT_seasonal = getIVT(original, seasonal)
print(f"Takes {(time.time()-count):.3f} sec")

Takes 71.845 sec


In [6]:
# Save IVT shape
print(IVT_original.shape)
config.update({"Var_Data_structure_IVT": IVT_original.shape})
with open(config_path, 'w') as outfile:
    json.dump(config, outfile, sort_keys=True)
    outfile.close()

(2, 43, 365, 60, 72)


In [7]:
# Save IVT
os.chdir(config["Path_IVT_calculation"])
np.savez(config["Fname_IVT"], IVT_original = IVT_original, IVT_seasonal = IVT_seasonal)